In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import strptime
import datetime
from datetime import datetime
import math
import warnings
warnings.filterwarnings('ignore')
import time
import csv
import glob
import logging
import importlib
import tweets_utils
import json
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import re
from operator import itemgetter
import matplotlib.cm as mcm
import community
from community import community_louvain as cm
import leidenalg as la
import igraph as ig
from cdlib import algorithms
import plotly.express as px
%autosave 3600

Autosaving every 3600 seconds


#### Generate user - rt (once in file is useless)

In [9]:
rt_user_name = []
user_name = []
result = []
files_tweets = glob.glob("F:/COVID-19-Tweets/*/tweets.csv")
cols_tweets = [u'user_screen_name', u"rt_created_at",u"rt_user_screen_name"]
importlib.reload(tweets_utils)
start_time = time.perf_counter()
j = 0
for filename in files_tweets:
    print(f"Reading file: {filename}")
    result.extend(tweets_utils.process_all_data(filename, cols_tweets, True))
    lista = result[j]
    j = j + 1
    print(f"Mese numero {j}")
    for i in range(len(lista)):
        #user who retweet
        user_name.extend(lista[i]["retweet_users"])
        #user retweeted
        rt_user_name.extend(lista[i]["retweeted_users"])
stop_time = time.perf_counter()
print("Time: ",stop_time-start_time)

Reading file: F:/COVID-19-Tweets\2020-01\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Mese numero 1
Reading file: F:/COVID-19-Tweets\2020-02\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Mese numero 2
Reading file: F:/COVID-19-Tweets\2020-03\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7
Processing chunk 8
Processing chunk 9
Processing chunk 10
Processing chunk 11
Processing chunk 12
Processing chunk 13
Processing chunk 14
Processing chunk 15
Mese numero 3
Reading file: F:/COVID-19-Tweets\2020-04\tweets.csv
Processing chunk 1
Processing chunk 2
Processing chunk 3
Processing chunk 4
Processing chunk 5
Processing chunk 6
Processing chunk 7


In [13]:
df_rt = pd.DataFrame(list(zip(user_name, rt_user_name)), columns =['name', 'rt_name'])

In [14]:
del user_name
del rt_user_name

In [15]:
df_rt.to_csv("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/large_files/user_rtuser.csv", index=False)

In [2]:
df_rt = pd.read_csv("C:/Users/Gianluca/Desktop/Supsi/Git/BachelorProject/large_files/user_rtuser.csv")

In [16]:
df_rt

,name,rt_name
0,Huerconetzin,AnneKPIX
1,JustAnotherAme4,cnni
2,HHSRegion8,CDCgov
3,Paxman42,OurWarOnCancer
4,beerhowell,nytimes
...,...,...
48312499,Dinora21971394,saludhn
48312500,EllyJonkerVliet,StopEcocideNL
48312501,anna76746180,Haggis_UK
48312502,SueHealy16,Auchindrain


### Network

In [ ]:
retweets_graph = nx.from_pandas_edgelist(df_rt,'name','rt_name', create_using=nx.DiGraph())
print(nx.info(retweets_graph))

In [ ]:
isolated_nodes = list(nx.isolates(retweets_graph))
print('Isoleted nodes:',len(isolated_nodes))

In [ ]:
print('Strongly connected: ',nx.is_strongly_connected(retweets_graph))

In [ ]:
print('Number of connected components: ',nx.number_strongly_connected_components(retweets_graph))

In [ ]:
# Extract the giant connected component
Gcc = sorted((retweets_graph.subgraph(c) for c in nx.strongly_connected_components(retweets_graph)), key = len,reverse=True)[0]
print("Number of nodes in the Gcc graph: ")
print(len(Gcc.nodes()))
print("Number of edges in the Gcc graph: ")
print(len(Gcc.edges()))

In [ ]:
degrees = [val for (node, val) in retweets_graph.degree()]

print(f"The maximum degree of the Graph is {np.max(degrees)}")   
print(f"The minimum degree of the Graph is {np.min(degrees)}")

out_degrees = [val for (node, val) in retweets_graph.out_degree()]

print(f"The maximum out_degree of the Graph is {np.max(out_degrees)}")   
print(f"The minimum out_degree of the Graph is {np.min(out_degrees)}")

in_degrees = [val for (node, val) in retweets_graph.in_degree()]

print(f"The maximum in_degree of the Graph is {np.max(in_degrees)}")   
print(f"The minimum in_degree of the Graph is {np.min(in_degrees)}")

In [ ]:
density = nx.density(retweets_graph)
print("Network density:", density)

In [ ]:
degree_dict = dict(retweets_graph.degree(retweets_graph.nodes()))
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [ ]:
n=5
print("Top n nodes by degree:")
for d in sorted_degree[:n]:
    print(f"the node with name {d[0]} , has a degree of {d[1]}")

In [ ]:
in_degree_dict = dict(retweets_graph.in_degree(retweets_graph.nodes()))
sorted_in_degree = sorted(in_degree_dict.items(), key=itemgetter(1), reverse=True)

In [ ]:
n=5
print("Top n nodes by in_degree:")
for d in sorted_in_degree[:n]:
    print(f"the node with name {d[0]} , has a in-degree of {d[1]}")

In [ ]:
out_degree_dict = dict(retweets_graph.out_degree(retweets_graph.nodes()))
sorted_out_degree = sorted(out_degree_dict.items(), key=itemgetter(1), reverse=True)

In [ ]:
n=5
print("Top n nodes by out_degree:")
for d in sorted_out_degree[:n]:
    print(f"the node with name {d[0]} , has a out-degree of {d[1]}")

In [ ]:
graph_centrality = nx.degree_centrality(retweets_graph)
max_de = max(graph_centrality.items(), key=itemgetter(1))
graph_closeness = nx.closeness_centrality(retweets_graph)
max_clo = max(graph_closeness.items(), key=itemgetter(1))
graph_betweenness = nx.betweenness_centrality(retweets_graph, normalized=True, endpoints=False)
max_bet = max(graph_betweenness.items(), key=itemgetter(1))

print(f"the node with id {max_de[0]} has a degree centrality of {max_de[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_clo[0]} has a closeness centrality of {max_clo[1]:.2f} which is the maximum of the Graph")
print(f"the node with id {max_bet[0]} has a betweenness centrality of {max_bet[1]:.2f} which is the maximum of the Graph")

In [ ]:
#pageRank centrality for each node
pagerank_centr = nx.pagerank(retweets_graph)

#### Communities with Louvain modularity

In [ ]:
retweets_graph_und = nx.from_pandas_edgelist(df_rt,'name','rt_name')
print(nx.info(retweets_graph_und))

In [ ]:
best_part = cm.best_partition(retweets_graph_und, random_state=42)

In [ ]:
size = float(len(set(best_part.values())))
print("Number of communities: ", size)

In [ ]:
#check that sum of each community size = number of nodes
communities_size = np.zeros(len(set(best_part.values())))
for v in list(best_part.values()):
    communities_size[v] +=1
print(sum(communities_size))

In [ ]:
print("Louvain Modularity: ",cm.modularity(best_part, retweets_graph_und))

In [ ]:
# color the nodes according to their partition
pos = nx.spring_layout(retweets_graph_und)

plt.figure(figsize=(10,10)) 
cmap = mcm.get_cmap('viridis', max(best_part.values()) + 1)
shapes = 'so^>v<dph8'
shapes_iter = 0
for node, color in best_part.items():
    nx.draw_networkx_nodes(retweets_graph_und, pos, [node], node_size=40,
                           node_color=[cmap.colors[color]],
                           node_shape=shapes[shapes_iter])
    shapes_iter +=1
    if shapes_iter == len(shapes):
        shapes_iter=0
nx.draw_networkx_edges(retweets_graph_und, pos, alpha=0.5)
#plt.savefig('plots/communities')
plt.show() #add labels

#### Communities with Leiden

In [ ]:
tuples = [tuple(x) for x in df_rt.values]
retweets_graph = ig.Graph.TupleList(tuples, directed = True)

In [ ]:
n_vertices = retweets_graph.vcount()
print("Number of vertices:", n_vertices)
print("Number of edges:", retweets_graph.ecount())
print("Density of the graph:", 2*retweets_graph.ecount()/(retweets_graph.vcount()*(retweets_graph.vcount()-1)))

In [ ]:
total_degree=0
total_indegree = 0
total_outdegree = 0
for n in range(n_vertices):
    total_degree += retweets_graph.degree(n)
    total_indegree += retweets_graph.indegree(n)
    total_outdegree += retweets_graph.outdegree(n)
print("Average indegree:", total_indegree/n_vertices)
print("Average indegree:", total_outdegree/n_vertices)

In [ ]:
partition = la.find_partition(retweets_graph, la.ModularityVertexPartition)
partition

In [ ]:
retweets_graph.vs[0]['name']

In [ ]:
retweets_graph.vs()["name"]

In [ ]:
partition.membership

In [ ]:
zip_iterator = zip(retweets_graph.vs()["name"], partition.membership)
best_part = dict(zip_iterator)
best_part